In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May  7 04:06:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!git clone https://github.com/salaniz/pycocoevalcap.git
%cd pycocoevalcap
!pip install .
%cd ..


Cloning into 'pycocoevalcap'...
remote: Enumerating objects: 821, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 821 (delta 4), reused 3 (delta 3), pack-reused 809 (from 2)
Receiving objects: 100% (821/821), 130.06 MiB | 50.64 MiB/s, done.
Resolving deltas: 100% (424/424), done.
/content/pycocoevalcap
Processing /content/pycocoevalcap
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=97150f49da2a49ac7f54cb9c585f2f19123ba3ca09fce2c2325373241338d232
  Stored in directory: /tmp/pip-ephem-wheel-cache-z6bnfjgd/wheels/0e/98/9f/b6578f2310a0adf702387edf950a2ba69dbf680c0b6830b312
Successfully built pycocoevalcap
/content


# **Blip model(Base code)**

In [ ]:
# Install Hugging Face Transformers (for BLIP-2) and other libraries
!pip install transformers
!pip install torch torchvision
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install git+https://github.com/jmhessel/clipscore.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch, os, json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import evaluate

In [ ]:
# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load BLIP processor and model (using the 'blip-image-captioning-base' model)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device).eval()

# Set paths to the image and annotation directories
images_path = "/content/drive/MyDrive/val2017/"
annotations_path = "/content/drive/MyDrive/annotations/captions_val2017.json"

# Load annotations
with open(annotations_path, "r") as f:
    coco_data = json.load(f)

image_id_to_filename = {img["id"]: img["file_name"] for img in coco_data["images"]}
gt_captions = {}
for ann in coco_data["annotations"]:
    img_id = ann["image_id"]
    if img_id not in gt_captions:
        gt_captions[img_id] = []
    gt_captions[img_id].append(ann["caption"])


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
# Filter valid images
image_ids = list(gt_captions.keys())[:2000]  # Limit to 2000 for your dataset
image_files = [image_id_to_filename[i] for i in image_ids]

# Generate captions
results = []
gen_caps = []
ref_caps = []
ann_id = 0

for idx, (img_id, img_file) in enumerate(tqdm(zip(image_ids, image_files), total=len(image_ids))):
    img_path = os.path.join(images_path, img_file)
    image = Image.open(img_path).convert("RGB")

    # Preprocess and generate captions
    inputs = processor(images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    # Save results
    references = gt_captions[img_id]
    results.append({
        "Image_File": img_file,
        "Generated_Caption": caption,
        "Ground_Truth_Captions": references
    })
    gen_caps.append({"image_id": idx, "caption": caption})
    for ref in references:
        ref_caps.append({"image_id": idx, "id": ann_id, "caption": ref})
        ann_id += 1


100%|██████████| 2000/2000 [19:18<00:00,  1.73it/s]


In [ ]:
# Save results to CSV
#pd.DataFrame(results).to_csv("/content/drive/MyDrive/generated_results_baseline.csv", index=False)

In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm
import tempfile  # Import tempfile module
import json
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

# COCO-style Evaluation
pred_file = tempfile.NamedTemporaryFile(delete=False, suffix='.json').name
ref_file = tempfile.NamedTemporaryFile(delete=False, suffix='.json').name

with open(pred_file, "w") as f:
    json.dump(gen_caps, f)

image_list = [{"id": i} for i in range(len(image_ids))]
with open(ref_file, "w") as f:
    json.dump({"annotations": ref_caps, "images": image_list}, f)

coco = COCO(ref_file)
res = coco.loadRes(pred_file)
cocoEval = COCOEvalCap(coco, res)
cocoEval.evaluate()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.
computing Bleu score...
{'testlen': 14419, 'reflen': 17704, 'guess': [14419, 12419, 10419, 8419], 'correct': [11716, 6573, 3231, 1510]}
ratio: 0.814448712155399
Bleu_1: 0.647
Bleu_2: 0.522
Bleu_3: 0.407
Bleu_4: 0.313
computing METEOR score...
METEOR: 0.241
computing Rouge score...
ROUGE_L: 0.535
computing CIDEr score...
CIDEr: 1.011
computing SPICE score...
SPICE: 0.184


In [ ]:
# Print evaluation metrics
metrics = {k: round(v, 4) for k, v in cocoEval.eval.items()}
print(metrics)

{'Bleu_1': 0.647, 'Bleu_2': 0.5222, 'Bleu_3': 0.4068, 'Bleu_4': 0.3131, 'METEOR': 0.2407, 'ROUGE_L': np.float64(0.5353), 'CIDEr': np.float64(1.0105), 'SPICE': np.float64(0.1841)}


In [ ]:
#testing

In [ ]:
# Set device to CPU
device = "cpu"

# Load BLIP-1 model (Image Captioning model)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device).eval()

# Path to the testing folder containing your images
testing_images_path = "/content/drive/MyDrive/test2017"

# Get all image files in the testing folder (assuming .jpg or .jpeg format)
image_files = [f for f in os.listdir(testing_images_path) if f.endswith('.jpg') or f.endswith('.jpeg')]

# Generate captions
results = []

for idx, img_file in enumerate(tqdm(image_files, total=len(image_files))):
    img_path = os.path.join(testing_images_path, img_file)
    image = Image.open(img_path).convert("RGB")

    # Preprocess image and generate caption
    inputs = processor(images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    # Save results
    results.append({
        "Image_File": img_file,
        "Generated_Caption": caption
    })

# Save results as a CSV file
generated_results_df = pd.DataFrame(results)
generated_results_df.to_csv("/content/drive/MyDrive/generated_testing_results_blip1.csv", index=False)

# Optionally, you can print the generated captions
for result in results:
    print(f"Image: {result['Image_File']}")
    print(f"Generated Caption: {result['Generated_Caption']}")
    print("-" * 50)

100%|██████████| 30/30 [00:32<00:00,  1.08s/it]

Image: 000000047056.jpg
Generated Caption: a table with a vase and a vase of flowers
--------------------------------------------------
Image: 000000054498.jpg
Generated Caption: a man riding a motorcycle
--------------------------------------------------
Image: 000000102209.jpg
Generated Caption: a man is playing with a frc
--------------------------------------------------
Image: 000000040965.jpg
Generated Caption: a tray of food
--------------------------------------------------
Image: 000000026680.jpg
Generated Caption: two zebras walking in a field
--------------------------------------------------
Image: 000000181543.jpg
Generated Caption: a surfer is riding a wave in the ocean
--------------------------------------------------
Image: 000000241446.jpg
Generated Caption: a teddy bear sitting on a rail
--------------------------------------------------
Image: 000000023271.jpg
Generated Caption: two women in a field
--------------------------------------------------
Image: 000000488

# **Finetuning Blip Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#NEW
import os
import json
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from torch.utils.data import Dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import tempfile
import torch
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import random

# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set paths to the data directories
train_images_path = "/content/drive/MyDrive/train2017/"
train_captions_path = "/content/drive/MyDrive/annotations/captions_train2017.json"
val_images_path = "/content/drive/MyDrive/val2017/"
val_captions_path = "/content/drive/MyDrive/annotations/captions_val2017.json"
output_dir = "/content/drive/MyDrive/finetuned/final_model"

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Load the training annotations
with open(train_captions_path, "r") as f:
    train_coco_captions = json.load(f)

# Load the validation annotations
with open(val_captions_path, "r") as f:
    val_coco_captions = json.load(f)

In [ ]:
# Map image IDs to filenames and captions for training data
train_image_id_to_filename = {img['id']: img['file_name'] for img in train_coco_captions['images']}
train_gt_captions = {}
for annot in train_coco_captions['annotations']:
    image_id = annot['image_id']
    caption = annot['caption']
    if image_id not in train_gt_captions:
        train_gt_captions[image_id] = []
    train_gt_captions[image_id].append(caption)

# Map image IDs to filenames and captions for validation data
val_image_id_to_filename = {img['id']: img['file_name'] for img in val_coco_captions['images']}
val_gt_captions = {}
for annot in val_coco_captions['annotations']:
    image_id = annot['image_id']
    caption = annot['caption']
    if image_id not in val_gt_captions:
        val_gt_captions[image_id] = []
    val_gt_captions[image_id].append(caption)

In [ ]:
# Filter valid image files with captions for training
train_image_files = sorted([f for f in os.listdir(train_images_path) if f.endswith(".jpg")])
train_valid_image_files, train_valid_image_ids = [], []
for img_id, captions in train_gt_captions.items():
    filename = train_image_id_to_filename.get(img_id)
    if filename and filename in train_image_files:
        train_valid_image_files.append(filename)
        train_valid_image_ids.append(img_id)

# Filter valid image files with captions for validation
val_image_files = sorted([f for f in os.listdir(val_images_path) if f.endswith(".jpg")])
val_valid_image_files, val_valid_image_ids = [], []
for img_id, captions in val_gt_captions.items():
    filename = val_image_id_to_filename.get(img_id)
    if filename and filename in val_image_files:
        val_valid_image_files.append(filename)
        val_valid_image_ids.append(img_id)

In [ ]:
# Define Dataset Class for Training and Validation
class CocoCaptionDataset(Dataset):
    def __init__(self, image_dir, image_ids, image_filenames, gt_captions, processor):
        self.image_dir = image_dir
        self.image_ids = image_ids
        self.image_filenames = image_filenames
        self.gt_captions = gt_captions
        self.processor = processor

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_file = self.image_filenames[idx]
        img_path = os.path.join(self.image_dir, img_file)
        image = Image.open(img_path).convert("RGB")
        caption = random.choice(self.gt_captions[img_id])
        inputs = self.processor(images=image, text=caption, return_tensors="pt", padding="max_length", truncation=True, max_length=30)
        inputs = {k: v.squeeze() for k, v in inputs.items()}
        inputs["labels"] = inputs["input_ids"]
        return inputs


In [ ]:
# Load the pre-trained model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
# Prepare the datasets
train_dataset = CocoCaptionDataset(train_images_path, train_valid_image_ids, train_valid_image_files, train_gt_captions, processor)
val_dataset = CocoCaptionDataset(val_images_path, val_valid_image_ids, val_valid_image_files, val_gt_captions, processor)

In [ ]:
# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=16,  # You can increase if GPU memory allows
    per_device_eval_batch_size=32,   # Large eval batch size for inference
    num_train_epochs=3,
    learning_rate=3e-5,              # Good learning rate for fine-tuning
    warmup_steps=500,                # Stabilize training with warmup
    save_steps=1000,                 # Save checkpoints every 1000 steps
    logging_steps=100,               # Log every 100 steps
    eval_steps=500,                  # Evaluate every 500 steps (adjust if necessary)
    eval_strategy="steps",     # Evaluate after certain steps
    save_total_limit=2,              # Limit number of saved checkpoints
    fp16=True,                       # Mixed precision training for faster and more memory-efficient training
    gradient_accumulation_steps=1,   # No need for gradient accumulation unless memory is an issue
    dataloader_num_workers=4,       # Adjust based on available CPU cores
    remove_unused_columns=True,      # Clean up unused columns in the dataset
    predict_with_generate=True,      # Ensure generation for image captioning tasks
)

# Trainer Setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor,
)

<ipython-input-23-6566318c9472>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Train the model
trainer.train()
print("Training completed.")

Step,Training Loss,Validation Loss
500,0.718500,0.856827
1000,0.778900,0.826860
1500,0.702700,0.804517


Training completed.


In [ ]:
# Save the trained model and processor
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

[]

In [ ]:
import torch

model_path = os.path.join(output_dir, 'pytorch_model.bin')
torch.save(model.state_dict(), model_path)
print(f"Model weights saved to {model_path}")

Model weights saved to /content/drive/MyDrive/finetuned/final_model/pytorch_model.bin


In [ ]:
# Evaluate on the validation set
results = []
gen_caps = []
ref_caps = []
ann_id = 0
num_samples = min(2000, len(val_valid_image_files))  # Adjust the number of samples
for idx in tqdm(range(num_samples)):
    img_file = val_valid_image_files[idx]
    img_id = val_valid_image_ids[idx]
    img_path = os.path.join(val_images_path, img_file)
    image = Image.open(img_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    output = model.generate(**inputs)
    caption = processor.decode(output[0], skip_special_tokens=True)
    references = val_gt_captions[img_id]

    results.append({
        "Image_File": img_file,
        "Generated_Caption": caption,
        "Ground_Truth_Captions": references
    })
    gen_caps.append({"image_id": idx, "caption": caption})
    for ref in references:
        ref_caps.append({"image_id": idx, "id": ann_id, "caption": ref})
        ann_id += 1


100%|██████████| 2000/2000 [09:14<00:00,  3.61it/s]


In [ ]:
# Save results to CSV
df = pd.DataFrame(results)
df.to_csv("/content/drive/MyDrive/generated_eval_results.csv", index=False)


In [ ]:
# Run COCOEvalCap for evaluation
pred_file = tempfile.NamedTemporaryFile(delete=False, suffix='.json').name
ref_file = tempfile.NamedTemporaryFile(delete=False, suffix='.json').name
with open(pred_file, "w") as f:
    json.dump(gen_caps, f)
image_ids = list({ann["image_id"] for ann in ref_caps})
image_list = [{"id": i} for i in image_ids]
with open(ref_file, "w") as f:
    json.dump({"annotations": ref_caps, "images": image_list}, f)

coco = COCO(ref_file)
res = coco.loadRes(pred_file)
cocoEval = COCOEvalCap(coco, res)
cocoEval.evaluate()

# Print evaluation scores
scores = cocoEval.eval
scores = {k: round(v, 4) for k, v in scores.items()}
print(scores)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 19851, 'reflen': 19523, 'guess': [19851, 17851, 15851, 13851], 'correct': [15264, 8677, 4390, 2185]}
ratio: 1.0168006966141978
Bleu_1: 0.769
Bleu_2: 0.611
Bleu_3: 0.470
Bleu_4: 0.357
computing METEOR score...
METEOR: 0.287
computing Rouge score...
ROUGE_L: 0.575
computing CIDEr score...
CIDEr: 1.227
computing SPICE score...
SPICE: 0.220
{'Bleu_1': 0.7689, 'Bleu_2': 0.6114, 'Bleu_3': 0.4695, 'Bleu_4': 0.3575, 'METEOR': 0.2869, 'ROUGE_L': np.float64(0.5754), 'CIDEr': np.float64(1.2265), 'SPICE': np.float64(0.2202)}


In [ ]:
import os
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

# Define the path to the testing folder
testing_images_path = "/content/drive/MyDrive/test2017"  # Replace with the actual path to your testing folder

# Load the fine-tuned model and processor
model = BlipForConditionalGeneration.from_pretrained(output_dir).to(device)
processor = BlipProcessor.from_pretrained(output_dir)

# List all image files in the testing folder
image_files = [f for f in os.listdir(testing_images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Function to generate caption for a single image
def generate_caption(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)

    # Generate a caption
    output = model.generate(**inputs)
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

# Iterate over the images in the testing folder and generate captions
for img_file in image_files:
    img_path = os.path.join(testing_images_path, img_file)
    caption = generate_caption(img_path)
    print(f"Caption for {img_file}: {caption}")


Caption for 000000023271.jpg: two women with umbrellas walk down a dirt road.
Caption for 000000505458.jpg: a close up of a vegetable dish with carrots and broccoli.
Caption for 000000526327.jpg: a man in a blue jacket is snowboarding on a ramp.
Caption for 000000488045.jpg: a bench sitting in the snow near a lake.
Caption for 000000471220.jpg: a man and a woman playing a video game.
Caption for 000000526846.jpg: a woman is walking past a display of furniture.
Caption for 000000509670.jpg: a yellow vase and a yellow candle on a table.
Caption for 000000557330.jpg: a man holding a tennis racket on a court.
Caption for 000000468714.jpg: a busy city street with a lot of traffic.
Caption for 000000326419.jpg: a red and white train traveling down the tracks.
Caption for 000000379473.jpg: a bird is sitting on a branch with a sky background.
Caption for 000000304597.jpg: a man riding a surfboard on top of a wave.
Caption for 000000114063.jpg: a small toilet with a blue bucket next to it.
Capt

# **Blip 2 model (Salesforce/blip2-flan-t5-xl-coco) Finetuned on coco**

In [ ]:
!pip install transformers accelerate datasets
!pip install pycocotools pycocoevalcap

import os, json, tempfile
from PIL import Image
import torch
import pandas as pd
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
from torch.utils.data import Dataset


device = "cuda" if torch.cuda.is_available() else "cpu"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install git+https://github.com/salesforce/LAVIS.git --no-deps

ERROR: Operation cancelled by user
^C


In [ ]:
!pip install iopath
!pip install decord



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=7c617a4b8172f09ada20ebb97d7dd889cc3dabb34134eb7984d47d5a6be22df0
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built iopath
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 116.2 MB/s eta 0:00:00


In [ ]:
!pip install webdataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers accelerate datasets
!pip install pycocotools pycocoevalcap

In [ ]:
!git clone https://github.com/salaniz/pycocoevalcap.git
%cd pycocoevalcap
!pip install .
%cd ..


fatal: destination path 'pycocoevalcap' already exists and is not an empty directory.
/content/pycocoevalcap
Processing /content/pycocoevalcap
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=593642242 sha256=46d7cc1635732b07fae784a422d5bad77b7db803ac418c55073b9a7bdceca659
  Stored in directory: /tmp/pip-ephem-wheel-cache-rag0_zww/wheels/0e/98/9f/b6578f2310a0adf702387edf950a2ba69dbf680c0b6830b312
Successfully built pycocoevalcap
  Attempting uninstall: pycocoevalcap
    Found existing installation: pycocoevalcap 1.2
    Uninstalling pycocoevalcap-1.2:
      Successfully uninstalled pycocoevalcap-1.2
/content


In [ ]:
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import os
import json
import pandas as pd
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import tempfile

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load BLIP-2 FLAN-T5-XL
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl-coco")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl-coco", device_map="auto", torch_dtype=torch.float16).eval()


images_path = "/content/drive/MyDrive/val2017/"
annotations_path = "/content/drive/MyDrive/annotations/captions_val2017.json"

# Load annotations
with open(annotations_path, "r") as f:
    coco_data = json.load(f)

image_id_to_filename = {img["id"]: img["file_name"] for img in coco_data["images"]}
gt_captions = {}
for ann in coco_data["annotations"]:
    img_id = ann["image_id"]
    if img_id not in gt_captions:
        gt_captions[img_id] = []
    gt_captions[img_id].append(ann["caption"])

# Filter valid images
image_ids = list(gt_captions.keys())[:2000]  # limit to 100 for demo
image_files = [image_id_to_filename[i] for i in image_ids]

# Generate captions
results = []
gen_caps = []
ref_caps = []
ann_id = 0

for idx, (img_id, img_file) in enumerate(tqdm(zip(image_ids, image_files), total=len(image_ids))):
    img_path = os.path.join(images_path, img_file)
    image = Image.open(img_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    # Save results
    references = gt_captions[img_id]
    results.append({
        "Image_File": img_file,
        "Generated_Caption": caption,
        "Ground_Truth_Captions": references
    })
    gen_caps.append({"image_id": idx, "caption": caption})
    for ref in references:
        ref_caps.append({"image_id": idx, "id": ann_id, "caption": ref})
        ann_id += 1

# Save CSV
pd.DataFrame(results).to_csv("/content/drive/MyDrive/generated_results.csv", index=False)

# COCO-style Evaluation
pred_file = tempfile.NamedTemporaryFile(delete=False, suffix='.json').name
ref_file = tempfile.NamedTemporaryFile(delete=False, suffix='.json').name

with open(pred_file, "w") as f:
    json.dump(gen_caps, f)

image_list = [{"id": i} for i in range(len(image_ids))]
with open(ref_file, "w") as f:
    json.dump({"annotations": ref_caps, "images": image_list}, f)

coco = COCO(ref_file)
res = coco.loadRes(pred_file)
cocoEval = COCOEvalCap(coco, res)
cocoEval.evaluate()

# Print scores
metrics = {k: round(v, 4) for k, v in cocoEval.eval.items()}
print(metrics)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [29:50<00:00,  1.12it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 19822, 'reflen': 19562, 'guess': [19822, 17822, 15822, 13822], 'correct': [14774, 8217, 4016, 1869]}
ratio: 1.0132910745322046
Bleu_1: 0.745
Bleu_2: 0.586
Bleu_3: 0.443
Bleu_4: 0.330
computing METEOR score...
METEOR: 0.272
computing Rouge score...
ROUGE_L: 0.560
computing CIDEr score...
CIDEr: 1.118
computing SPICE score...
SPICE: 0.216
{'Bleu_1': 0.7453, 'Bleu_2': 0.5862, 'Bleu_3': 0.4435, 'Bleu_4': 0.3295, 'METEOR': 0.2725, 'ROUGE_L': np.float64(0.5598), 'CIDEr': np.float64(1.1175), 'SPICE': np.float64(0.2159)}


In [ ]:
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm

# Set device to CPU
device = "cpu"

# Load BLIP-2 FLAN-T5-XL model
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl-coco")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl-coco").to(device).eval()

# Path to the testing folder containing your images
testing_images_path = "/content/drive/MyDrive/test2017"

# Get all image files in the testing folder (assuming .jpg or .jpeg format)
image_files = [f for f in os.listdir(testing_images_path) if f.endswith('.jpg') or f.endswith('.jpeg')]

# Generate captions
results = []

for idx, img_file in enumerate(tqdm(image_files, total=len(image_files))):
    img_path = os.path.join(testing_images_path, img_file)
    image = Image.open(img_path).convert("RGB")

    # Preprocess image and generate caption
    inputs = processor(images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    # Save results
    results.append({
        "Image_File": img_file,
        "Generated_Caption": caption
    })

# Save results as a CSV file
generated_results_df = pd.DataFrame(results)
generated_results_df.to_csv("/content/drive/MyDrive/generated_testing_results.csv", index=False)

# Optionally, you can print the generated captions
for result in results:
    print(f"Image: {result['Image_File']}")
    print(f"Generated Caption: {result['Generated_Caption']}")
    print("-" * 50)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 30/30 [02:58<00:00,  5.93s/it]

Image: 000000023271.jpg
Generated Caption: two women in dresses walking in a field with an umbrella
--------------------------------------------------
Image: 000000505458.jpg
Generated Caption: a bowl of vegetables with broccoli, carrots, and zucchini
--------------------------------------------------
Image: 000000526327.jpg
Generated Caption: a person on a snowboard is riding down a ramp
--------------------------------------------------
Image: 000000488045.jpg
Generated Caption: a bench is sitting on a snowy path near a lake
--------------------------------------------------
Image: 000000471220.jpg
Generated Caption: a man and woman are playing a video game
--------------------------------------------------
Image: 000000526846.jpg
Generated Caption: a group of people walking around a display
--------------------------------------------------
Image: 000000509670.jpg
Generated Caption: a vase with a flower and a plant on top of a wooden box
---------------------------------------------

# **Blip 2 (2.7b model)**

In [ ]:
pip install torch transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration


In [ ]:
# Load BLIP-2 7B model and processor
import torch
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", device_map="auto", torch_dtype=torch.float16).eval()


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
import json


In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"

images_path = "/content/drive/MyDrive/val2017/"
annotations_path = "/content/drive/MyDrive/annotations/captions_val2017.json"

# Load annotations
# Load annotations
with open(annotations_path, "r") as f:
    coco_data = json.load(f)

image_id_to_filename = {img["id"]: img["file_name"] for img in coco_data["images"]}
gt_captions = {}
for ann in coco_data["annotations"]:
    img_id = ann["image_id"]
    if img_id not in gt_captions:
        gt_captions[img_id] = []
    gt_captions[img_id].append(ann["caption"])

# Filter valid images
image_ids = list(gt_captions.keys())[:2000]  # limit to 2000 for your dataset
image_files = [image_id_to_filename[i] for i in image_ids]



In [ ]:
# Generate captions
from tqdm import tqdm  # Add this line to import tqdm
from PIL import Image
import os

results = []
gen_caps = []
ref_caps = []
ann_id = 0

for idx, (img_id, img_file) in enumerate(tqdm(zip(image_ids, image_files), total=len(image_ids))):
    img_path = os.path.join(images_path, img_file)
    image = Image.open(img_path).convert("RGB")

    # Preprocess and generate captions
    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    # Save results
    references = gt_captions[img_id]
    results.append({
        "Image_File": img_file,
        "Generated_Caption": caption,
        "Ground_Truth_Captions": references
    })
    gen_caps.append({"image_id": idx, "caption": caption})
    for ref in references:
        ref_caps.append({"image_id": idx, "id": ann_id, "caption": ref})
        ann_id += 1




100%|██████████| 2000/2000 [35:40<00:00,  1.07s/it]


NameError: name 'pd' is not defined

In [ ]:
!git clone https://github.com/salaniz/pycocoevalcap.git
%cd pycocoevalcap
!pip install .
%cd ..


Cloning into 'pycocoevalcap'...
remote: Enumerating objects: 821, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 821 (delta 4), reused 3 (delta 3), pack-reused 809 (from 2)
Receiving objects: 100% (821/821), 130.06 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (424/424), done.
/content/pycocoevalcap
Processing /content/pycocoevalcap
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=2d051328d71f9b1c7b5fc52d1175bcb3c0cad91f1632db8526a164259f517c49
  Stored in directory: /tmp/pip-ephem-wheel-cache-t8g9cjze/wheels/0e/98/9f/b6578f2310a0adf702387edf950a2ba69dbf680c0b6830b312
Successfully built pycocoevalcap
/content


In [ ]:
import pandas as pd
import tempfile
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

# Save results to CSV
pd.DataFrame(results).to_csv("/content/drive/MyDrive/generated_results.csv", index=False)

# COCO-style Evaluation
pred_file = tempfile.NamedTemporaryFile(delete=False, suffix='.json').name
ref_file = tempfile.NamedTemporaryFile(delete=False, suffix='.json').name

with open(pred_file, "w") as f:
    json.dump(gen_caps, f)

image_list = [{"id": i} for i in range(len(image_ids))]
with open(ref_file, "w") as f:
    json.dump({"annotations": ref_caps, "images": image_list}, f)

coco = COCO(ref_file)
res = coco.loadRes(pred_file)
cocoEval = COCOEvalCap(coco, res)
cocoEval.evaluate()

# Print evaluation metrics
metrics = {k: round(v, 4) for k, v in cocoEval.eval.items()}
print(metrics)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.
computing Bleu score...
{'testlen': 16550, 'reflen': 18242, 'guess': [16550, 14550, 12550, 10550], 'correct': [13705, 8112, 4155, 2052]}
ratio: 0.9072470123889427
Bleu_1: 0.748
Bleu_2: 0.613
Bleu_3: 0.483
Bleu_4: 0.375
computing METEOR score...
METEOR: 0.275
computing Rouge score...
ROUGE_L: 0.580
computing CIDEr score...
CIDEr: 1.251
computing SPICE score...
SPICE: 0.217
{'Bleu_1': 0.7476, 'Bleu_2': 0.6134, 'Bleu_3': 0.4827, 'Bleu_4': 0.3749, 'METEOR': 0.2747, 'ROUGE_L': np.float64(0.5805), 'CIDEr': np.float64(1.2514), 'SPICE': np.float64(0.2173)}


In [ ]:
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm

# Set device to CPU
device = "cpu"

# Load BLIP-2 FLAN-T5-XL model
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b").to(device).eval()

# Path to the testing folder containing your images
testing_images_path = "/content/drive/MyDrive/test2017"

# Get all image files in the testing folder (assuming .jpg or .jpeg format)
image_files = [f for f in os.listdir(testing_images_path) if f.endswith('.jpg') or f.endswith('.jpeg')]

# Generate captions
results = []

for idx, img_file in enumerate(tqdm(image_files, total=len(image_files))):
    img_path = os.path.join(testing_images_path, img_file)
    image = Image.open(img_path).convert("RGB")

    # Preprocess image and generate caption
    inputs = processor(images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    # Save results
    results.append({
        "Image_File": img_file,
        "Generated_Caption": caption
    })

# Save results as a CSV file
generated_results_df = pd.DataFrame(results)
generated_results_df.to_csv("/content/drive/MyDrive/generated_testing_results_blip2base.csv", index=False)

# Optionally, you can print the generated captions
for result in results:
    print(f"Image: {result['Image_File']}")
    print(f"Generated Caption: {result['Generated_Caption']}")
    print("-" * 50)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 30/30 [02:27<00:00,  4.93s/it]

Image: 000000047056.jpg
Generated Caption: a vase with flowers in it
--------------------------------------------------
Image: 000000054498.jpg
Generated Caption: a man riding a motorcycle in a parade
--------------------------------------------------
Image: 000000102209.jpg
Generated Caption: a man throwing a frisbee in a park
--------------------------------------------------
Image: 000000040965.jpg
Generated Caption: a tray of food on a plane
--------------------------------------------------
Image: 000000026680.jpg
Generated Caption: two zebras walking in a field with mountains in the background
--------------------------------------------------
Image: 000000181543.jpg
Generated Caption: a person riding a wave on a surfboard
--------------------------------------------------
Image: 000000241446.jpg
Generated Caption: a train is passing by
--------------------------------------------------
Image: 000000023271.jpg
Generated Caption: two women walking in the grass with an umbrella
---

In [ ]:
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm

# Set device to CPU
device = "cpu"

# Load BLIP-2 FLAN-T5-XL model
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b").to(device).eval()

# Path to the testing folder containing your images
testing_images_path = "/content/drive/MyDrive/testing"

# Get all image files in the testing folder (assuming .jpg or .jpeg format)
image_files = [f for f in os.listdir(testing_images_path) if f.endswith('.jpg') or f.endswith('.jpeg')]

# Generate captions
results = []

for idx, img_file in enumerate(tqdm(image_files, total=len(image_files))):
    img_path = os.path.join(testing_images_path, img_file)
    image = Image.open(img_path).convert("RGB")

    # Preprocess image and generate caption
    inputs = processor(images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    # Save results
    results.append({
        "Image_File": img_file,
        "Generated_Caption": caption
    })

# Save results as a CSV file
generated_results_df = pd.DataFrame(results)
generated_results_df.to_csv("/content/drive/MyDrive/generated_testing_results_blip2base_2.7b.csv", index=False)

# Optionally, you can print the generated captions
for result in results:
    print(f"Image: {result['Image_File']}")
    print(f"Generated Caption: {result['Generated_Caption']}")
    print("-" * 50)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

100%|██████████| 24/24 [01:36<00:00,  4.02s/it]

Image: 000000581886.jpg
Generated Caption: two people sitting on the beach with surfboards
--------------------------------------------------
Image: 000000581402.jpg
Generated Caption: a cat sleeping in a bowl
--------------------------------------------------
Image: 000000580975.jpg
Generated Caption: a row of motorcycles parked on the side of the road
--------------------------------------------------
Image: 000000581873.jpg
Generated Caption: a dog standing on a checkered floor
--------------------------------------------------
Image: 000000581422.jpg
Generated Caption: a man sitting on a bench next to a cow
--------------------------------------------------
Image: 000000581396.jpg
Generated Caption: a man taking a picture of himself in the mirror
--------------------------------------------------
Image: 000000581686.jpg
Generated Caption: two pigeons are sitting on a ledge
--------------------------------------------------
Image: 000000581702.jpg
Generated Caption: a bird with a re

# **CLIP Score calculation**

In [ ]:
!pip install transformers torch torchvision datasets sentencepiece clip-score
!pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import json
import os

class CocoEvalDataset(Dataset):
    def __init__(self, image_dir, annotation_path, num_samples=500):
        with open(annotation_path) as f:
            data = json.load(f)

        # Get first 2000 images
        self.image_ids = [img['id'] for img in data['images'][:num_samples]]
        self.image_files = {img['id']: img['file_name'] for img in data['images']}

        # Map image_id to captions
        self.gt_captions = {}
        for ann in data['annotations']:
            if ann['image_id'] in self.image_ids:
                if ann['image_id'] not in self.gt_captions:
                    self.gt_captions[ann['image_id']] = []
                self.gt_captions[ann['image_id']].append(ann['caption'])

        self.image_dir = image_dir

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.image_dir, self.image_files[img_id])
        image = Image.open(img_path).convert("RGB")
        captions = self.gt_captions[img_id]
        return image, captions, str(img_id)

# Initialize dataset
eval_dataset = CocoEvalDataset(
    image_dir="/content/drive/MyDrive/val2017",
    annotation_path="/content/drive/MyDrive/annotations/captions_val2017.json"
)

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install clip-score==0.1.0  # Specific working version

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_ovhuzit
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_ovhuzit
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Attempting uninstall: clip-score
    Found existing installation: clip-score 0.2.1
    Uninstalling clip-score-0.2.1:
      Successfully uninstalled clip-score-0.2.1


In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install torch torchvision

In [ ]:


import torch
import clip
from PIL import Image
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CLIP model
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

def calculate_clip_score(image, text):
    """Calculate CLIPScore between image and text"""
    try:
        # Preprocess image
        image_input = clip_preprocess(image).unsqueeze(0).to(device)

        # Tokenize text
        text_input = clip.tokenize([text]).to(device)

        # Extract features
        with torch.no_grad():
            image_features = clip_model.encode_image(image_input)
            text_features = clip_model.encode_text(text_input)

        # Normalize features
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        # Calculate similarity (original CLIPScore uses 2.5*max(0,cos_sim))
        similarity = (image_features @ text_features.T).item()
        return 2.5 * max(0, similarity)

    except Exception as e:
        print(f"Error calculating CLIPScore: {e}")
        return 0.0

def evaluate_clipscore(model, processor, dataset, num_samples=None):
    """Evaluate model on dataset using CLIPScore"""
    results = []
    model.eval()

    if num_samples:
        dataset = list(dataset)[:num_samples]  # Limit samples if specified

    for idx, (image, gt_captions, img_id) in enumerate(dataset):
        try:
            # Generate caption
            inputs = processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = model.generate(**inputs)
            pred_caption = processor.decode(outputs[0], skip_special_tokens=True)

            # Calculate CLIPScore
            score = calculate_clip_score(image, pred_caption)

            results.append({
                "image_id": img_id,
                "predicted_caption": pred_caption,
                "clip_score": score,
                "reference_captions": gt_captions
            })

            if (idx+1) % 50 == 0:
                print(f"Processed {idx+1} samples...")

        except Exception as e:
            print(f"Error processing sample {img_id}: {e}")
            continue

    avg_score = np.mean([r['clip_score'] for r in results]) if results else 0.0
    return avg_score, results

In [ ]:
from transformers import (
    BlipForConditionalGeneration,
    Blip2ForConditionalGeneration,
    AutoProcessor
)
import pandas as pd

models_to_evaluate = {
    "blip-base": "Salesforce/blip-image-captioning-base",
    "blip2-flan-t5-xl": "Salesforce/blip2-flan-t5-xl-coco",
    "blip2-opt-2.7b": "Salesforce/blip2-opt-2.7b"
}

results = []

for model_name, model_path in models_to_evaluate.items():
    print(f"\nEvaluating {model_name}...")

    # Load model
    if "blip2" in model_name:
        processor = AutoProcessor.from_pretrained(model_path)
        model = Blip2ForConditionalGeneration.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="auto"
        )
    else:
        processor = AutoProcessor.from_pretrained(model_path)
        model = BlipForConditionalGeneration.from_pretrained(model_path).to(device)

    # Evaluate
    avg_score, detailed_results = evaluate_clipscore(model, processor, eval_dataset)
    results.append({
        "model": model_name,
        "avg_clip_score": avg_score
    })

    # Save detailed results
    pd.DataFrame(detailed_results).to_csv(f"{model_name}_clipscore_results.csv", index=False)

    # Clean up
    del model, processor
    torch.cuda.empty_cache()

# Print summary
print("\nEvaluation Summary:")
print(pd.DataFrame(results))


Evaluating blip-base...
Processed 50 samples...
Processed 100 samples...
Processed 150 samples...
Processed 200 samples...
Processed 250 samples...
Processed 300 samples...
Processed 350 samples...
Processed 400 samples...
Processed 450 samples...
Processed 500 samples...

Evaluating blip2-flan-t5-xl...


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Processed 50 samples...
Processed 100 samples...
Processed 150 samples...
Processed 200 samples...
Processed 250 samples...
Processed 300 samples...
Processed 350 samples...
Processed 400 samples...
Processed 450 samples...
Processed 500 samples...

Evaluation Summary:
              model  avg_clip_score
0         blip-base        0.730167
1  blip2-flan-t5-xl        0.749786
2    blip2-opt-2.7b        0.759352


## **Clip Score calculation for fine-tuned model**


In [ ]:
from transformers import BlipForConditionalGeneration, AutoProcessor
import torch
from PIL import Image
import clip
import pandas as pd

# 1. Load your fine-tuned model
output_dir = "/content/drive/MyDrive/finetuned/final_model"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load processor and model
processor = AutoProcessor.from_pretrained(output_dir)
model = BlipForConditionalGeneration.from_pretrained(output_dir).to(device)
model.eval()

# 2. Load CLIP model for scoring
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

def calculate_clip_score(image, text):
    """Calculate CLIPScore between image and text"""
    try:
        image_input = clip_preprocess(image).unsqueeze(0).to(device)
        text_input = clip.tokenize([text]).to(device)

        with torch.no_grad():
            image_features = clip_model.encode_image(image_input)
            text_features = clip_model.encode_text(text_input)

        # Normalize features and calculate score
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (image_features @ text_features.T).item()
        return 2.5 * max(0, similarity)
    except Exception as e:
        print(f"Error calculating CLIPScore: {e}")
        return 0.0

# 3. Evaluation function for your fine-tuned model
def evaluate_finetuned_model(dataset, num_samples=None):
    results = []
    dataset = list(dataset)[:num_samples] if num_samples else dataset

    for idx, (image, gt_captions, img_id) in enumerate(dataset):
        try:
            # Generate caption
            inputs = processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = model.generate(**inputs)
            pred_caption = processor.decode(outputs[0], skip_special_tokens=True)

            # Calculate CLIPScore
            score = calculate_clip_score(image, pred_caption)

            results.append({
                "image_id": img_id,
                "predicted_caption": pred_caption,
                "clip_score": score,
                "reference_captions": gt_captions
            })

            if (idx+1) % 20 == 0:
                print(f"Processed {idx+1}/{len(dataset)} samples...")

        except Exception as e:
            print(f"Error processing {img_id}: {e}")
            continue

    return results

# 4. Run evaluation on your validation dataset
eval_results = evaluate_finetuned_model(eval_dataset, num_samples=500)  # Adjust sample size

# 5. Analyze results
avg_score = sum(r['clip_score'] for r in eval_results) / len(eval_results)
print(f"\nAverage CLIPScore: {avg_score:.4f}")

# Save detailed results
results_df = pd.DataFrame(eval_results)
results_df.to_csv("/content/drive/MyDrive/finetuned/clipscore_results.csv", index=False)
'''
# Optional: Compare with original model
print("\nComparing with original model...")
original_model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(device)
original_results = evaluate_finetuned_model(eval_dataset, num_samples=200)  # Same function works
original_avg = sum(r['clip_score'] for r in original_results) / len(original_results)
'''
print(f"\nFine-tuned model score: {avg_score:.4f}")
#print(f"Original model score: {original_avg:.4f}")
#print(f"Improvement: {avg_score - original_avg:+.4f}")

Processed 20/500 samples...
Processed 40/500 samples...
Processed 60/500 samples...
Processed 80/500 samples...
Processed 100/500 samples...
Processed 120/500 samples...
Processed 140/500 samples...
Processed 160/500 samples...
Processed 180/500 samples...
Processed 200/500 samples...
Processed 220/500 samples...
Processed 240/500 samples...
Processed 260/500 samples...
Processed 280/500 samples...
Processed 300/500 samples...
Processed 320/500 samples...
Processed 340/500 samples...
Processed 360/500 samples...
Processed 380/500 samples...
Processed 400/500 samples...
Processed 420/500 samples...
Processed 440/500 samples...
Processed 460/500 samples...
Processed 480/500 samples...
Processed 500/500 samples...

Average CLIPScore: 0.7715

Fine-tuned model score: 0.7715
